In [62]:
import pandas as pd
from pathlib import Path

data_folder = str(Path('.') / '../../input_data')
europe_data_path = str(Path(data_folder) / 'europe_data.xls')
europe_data_en_out_path = "europe_data_en.csv"
europe_data_de_out_path = "europe_data_de.csv"

In [63]:
df = pd.read_excel(europe_data_path, header=1, usecols=['lfd. Nummer', 'Titel', 'Zusammenfassung', 'Veröffentlichungs-Datum'])

In [64]:
df.rename(columns={'lfd. Nummer': 'id', 'Titel': 'title', 'Zusammenfassung': 'abstract', 'Veröffentlichungs-Datum': 'pub_date'}, inplace=True)

In [65]:
df.dropna(subset=['abstract'], inplace=True)

In [66]:
df[df.id == 9942]

,id,pub_date,title,abstract
9941,9942,06.01.2022,"[DE] SCHWEISSBRENNERSYSTEM MIT EINER WECHSELKUPPLUNG UND SCHWEISSBRENNER, BRENNERWECHSELSTATION, UND VERFAHREN ZUM AUFNEHMEN EINES SCHWEISSBRENNER EINES SCHWEISSBRENNERSYSTEM MIT EINEM SCHWEISSROBOTER ...","[DE] Um den Wechsel eines Schweißbrenners an einem Schweißroboter zu vereinfachen ist ein Schweißbrennersystem mit einer Wechselkupplung (2a) und einem Schweißbrenner (2b) vorgesehen, die voneinander gelöst werden können. An der Wechselkupplung (2a) sind ein erster Justierring (25a) federnd gelagert angeordnet und ein erstes Justierelement (32a) vorgesehen. Am Schweißbrenner (2b) sind ein zweiter Justierring (25b) und ein zweites Justierelement (32b) vorgesehen. Der erste Justierring (25a) ist bei von der Wechselkupplung (2a) gelöstem Schweißbrenner (2b) axial vom ersten Kupplungsende beabstandet angeordnet, sodass das erste Justierelement (32a) und das zweite Justierelement (32b) beim Verbinden des Schweißbrenners (2b) mit der Wechselkupplung (2a) zur Ausrichtung des Schweißbrenners (2b) relativ zur Wechselkupplung (2a) in Eingriff gelangen bevor Schweißbrennerkupplungsteile (16a, 16b) am Schweißbrenner (2b) und an der Wechselkupplung (2a) wechselwirken. <br /> [EN] In order to simplify ..."


In [67]:
from arrangement import arrange_txt_de, arrange_txt_en

In [68]:
mask_en = df['title'].str.contains(r'\[EN\]') & df['abstract'].str.contains(r'\[EN\]')
mask_de = df['title'].str.contains(r'\[DE\]') & df['abstract'].str.contains(r'\[DE\]')
df_en = df[mask_en].copy()
df_de = df[mask_de].copy()

In [69]:
df[mask_de & mask_en].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188 entries, 135 to 9943
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        188 non-null    int64 
 1   pub_date  188 non-null    object
 2   title     188 non-null    object
 3   abstract  188 non-null    object
dtypes: int64(1), object(3)
memory usage: 7.3+ KB


In [70]:
df_en[['title', 'abstract']] = df_en[['title', 'abstract']].apply(arrange_txt_en, axis=1)
df_de[['title', 'abstract']] = df_de[['title', 'abstract']].apply(arrange_txt_de, axis=1)

In [71]:
df_de['lang'] = 'de'
df_en['lang'] = 'en'
df = pd.concat([df_en, df_de], ignore_index=True)

In [72]:
df.dropna(subset=['abstract'], inplace=True)

In [73]:
df['lang'].value_counts() / len(df) * 100

en    58.873866
de    41.126134
Name: lang, dtype: float64

In [74]:
df['pub_date'] = pd.to_datetime(df['pub_date'], format='%d.%m.%Y')

In [75]:
df_filtered = df.sort_values(by='pub_date', ascending=False).drop_duplicates(subset='abstract')

In [76]:
df_filtered[df_filtered.duplicated('id', keep=False)].sort_values('id').tail(10)

,id,pub_date,title,abstract,lang
3370,9781,2022-01-20,CONTROL OF A CLEANING ROBOT,"A cleaning robot is configured to clean a predetermined floor area. A method for controlling the cleaning robot comprises steps of determining a section of the floor area, the treatment of which by the cleaning robot is intended to differ from the remaining floor area; deactivating a cleaning device of the cleaning robot; controlling the cleaning robot to move along a boundary ...",en
5786,9781,2022-01-20,STEUERUNG EINES REINIGUNGSROBOTERS,"Ein Reinigungsroboter ist dazu eingerichtet, eine vorbestimmte Bodenfläche zu reinigen. Ein Verfahren zum Steuern des Reinigungsroboters umfasst Schritte des Bestimmens eines Abschnitts der Bodenfläche, dessen Bearbeitung durch den Reinigungsroboter unterschiedlich von der restlichen Bodenfläche sein soll; des Deaktivierens einer Reinigungseinrichtung des Reinigungsroboters; des Steuerns des Reinigungsroboters, entlang einer Begrenzung des Abschnitts zu fahren; des Erfassens einer benutzergesteuerten Bestätigung der entlang gefahrenen Begrenzung; und des Freigebens der Reinigungseinrichtung. <br />",de
5807,9860,2022-01-13,ROBOTERZELLE ZUR HANDHABUNG VON STÜCKGUT,"Die vorliegende Erfindung betrifft eine Roboterzelle (1) zur Handhabung von Stückgut, welche folgendes umfasst: - eine Einhausung (2), - einen im Inneren der Einhausung (2) angeordneten Roboter (10), - einen ersten Aufnahmebereich (20) für einen ersten Stückgutbehälter (24), - einen zweiten Aufnahmebereich (22) für einen zweiten Stückgutbehälter (26), wobei der erste Aufnahmebereich (20) und der zweite Aufnahmebereich (22) ent- lang einer ersten Richtung (x) nebeneinander angeordnet sind, - eine Fördereinrichtung (40) für vereinzeltes Stückgut, welche durch die Einhausung (2) hindurchragt, und - einen Träger (50), an oder auf welchem der Roboter (10) entlang der ersten Richtung (x) beweglich gelagert ist. <br />",de
3396,9860,2022-01-13,ROBOT CELL FOR HANDLING PIECE GOODS,"The present invention relates to a robot cell (1) for handling piece goods, said robot cell comprising: - a housing (2), - a robot (10) arranged in the interior of the housing (2), - a first receiving region (20) for a first piece goods container (24), - a second ...",en
5809,9864,2022-01-13,"MOBILES, SELBSTFAHRENDES GERÄT","Es wird ein mobiles, selbstfahrendes Gerät (10), insbesondere Reinigungsgerät zur autonomen Bearbeitung von Bodenflächen wie ein Saug- und/oder Kehr- und/oder Wischroboter, angegeben, das ein Trägerteil, eine Steuerung und zumindest ein in oder an das Trägerteil ersetzbares Bauteil (2) aufweist. Das ersetzbare Bauteil (2) weist eine Identifikationsnummer auf, die an die Steuerung übertragbar ist, über die die Identifikationsnummer an eine externe Instanz (3) sendbar ist, um das einsetzbare Bauteil (2) von der externen Instanz (3) identifizieren zu lassen und Steuerungsparameter für das einsetzbare Bauteil (2) von der externen Instanz (3) an die Steuerung zu übertragen. Weiter wird ein Verfahren zur automatischen Bearbeitung von Bodenflächen mit Hilfe eines mobilen, selbstfahrenden Geräts (10) mit einem einsetzbaren Bauteil (2) und einer Steuerung angegeben. <br />",de
3399,9864,2022-01-13,"MOBILE, SELF-PROPELLED DEVICE","The invention relates to a mobile, self-propelled device (10), in particular a cleaning device for autonomously ...",en
3400,9865,2022-01-13,DETECTING PARAMETERS OF A ROBOT STRUCTURE,"The invention relates to a sensor module (100) for detecting parameters (F, p, a, T) of a robot structure (10) which has at least one manipulator (12) for positioning tasks. The sensor module (100) has: - at least one surface (111, 112, 113) which is designed to interact with the manipulator (12, 13) and - at least one sensor which is designed to ascertain a force (F) and/or a pressure (p) ...",en
5810,9865,2022-01-13,ERFASSEN VON PARAMETERN EINES ROBOTERAUFBAUS,"Zusammenfassend betrifft die Erfindung ein Sensormodul (100) zum Erfassen von Parametern (F, p, a, T)

In [77]:
# change pub_date to pub_year
# Convert pub_date to datetime format
# Extract the year from pub_date to create pub_year
df_filtered['pub_year'] = df_filtered['pub_date'].dt.year
df_filtered['text'] = df_filtered['title'] + '. ' + df_filtered['abstract']
df_filtered[['text']].head(1)

,text
3450,"Roboter mit hohlem Handgelenkelement. Roboter (10), umfassend:ein hohles erstes Handgelenkelement (26), das um eine erste Achsenlinie (A1) drehbar mit einem vorderen Ende eines Roboterarms (26) gekoppelt ist;ein hohles zweites Handgelenkelement (28), das um eine die erste Achsenlinie orthogonal kreuzende zweite Achsenlinie (A2) drehbar mit dem ersten Handgelenkelement gekoppelt ist;ein drittes Handgelenkelement (30), das um eine die zweite Achsenlinie orthogonal kreuzende dritte Achsenlinie (A3) mit dem zweiten Handgelenkelement gekoppelt ist und einen Endeffektor trägt, der eine Tätigkeit in Bezug auf ein Werkstück durchführt;einen ersten Leitungskörper (62), der so verlegt ist, dass er durch das Innere des ersten Handgelenkelements, des zweiten Handgelenkelements und des dritten Handgelenkelements verläuft und an den Endeffektor angeschlossen ist; undeinen zweiten Leitungskörper (64), der so verlegt ist, dass er durch das Innere des ersten Handgelenkelements und außerhalb des zweiten Handgelenkelements und des dritten ..."


In [78]:
df_filtered.drop(columns=['title', 'abstract'], inplace=True)

In [79]:
df_de = df_filtered[df_filtered['lang'] == 'de']
df_en = df_filtered[df_filtered['lang'] == 'en']

In [80]:
df_de.drop(columns=['lang'], inplace=True)
df_en.drop(columns=['lang'], inplace=True)

C:\Users\paul-\AppData\Local\Temp\ipykernel_22428\2259093244.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_de.drop(columns=['lang'], inplace=True)
C:\Users\paul-\AppData\Local\Temp\ipykernel_22428\2259093244.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en.drop(columns=['lang'], inplace=True)


In [81]:
df_de.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2243 entries, 3450 to 5842
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        2243 non-null   int64         
 1   pub_date  2243 non-null   datetime64[ns]
 2   pub_year  2243 non-null   int64         
 3   text      2243 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 87.6+ KB


In [82]:
df_en.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3215 entries, 0 to 3436
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        3215 non-null   int64         
 1   pub_date  3215 non-null   datetime64[ns]
 2   pub_year  3215 non-null   int64         
 3   text      3215 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 125.6+ KB


In [84]:
df_en[['id', 'pub_year', 'text']].to_csv(europe_data_en_out_path, index=False)
df_de[['id', 'pub_year', 'text']].to_csv(europe_data_de_out_path, index=False)